In [320]:
#Tokenization of Question
def tokenization(list_of_strings):
    BOW_Query=list_of_strings.split()
    return BOW_Query

In [321]:
#Removal of Stop words in the Question
def stop_words(strings_list):
    from nltk.corpus import stopwords
    stop = set(stopwords.words('english'))
    bow_query_sw=([i for i in strings_list if i not in stop])
    return bow_query_sw

In [322]:
#Stemming of the Question terms
def stemmer(strings_list):
    from nltk.stem import PorterStemmer
    query_words=[]
    ps = PorterStemmer()
    for w in strings_list:
        query_words.append(ps.stem(w))
    return query_words

In [323]:
#Removal of stop words in the answers
def sw_co(comments):
    from nltk.corpus import stopwords
    stop = set(stopwords.words('english'))
    comments_sw=[]
    for item in comments:
        stop_words=([i for i in item.lower().split() if i not in stop])
        comments_sw.append(stop_words)
    return comments_sw

In [324]:
#Stemming the words in the comments
def stemmer_co(strings):
    from nltk.stem import PorterStemmer
    comments_st=[]
    ps = PorterStemmer()
    for items in strings:
        comment_st_each=[]
        for w in items:
            comment_st_each.append(ps.stem(w))
        comments_st.append(comment_st_each)
    return comments_st

In [325]:
#Creating the unique words list from the above list.
#Creating a dictionary with word counter
def word_counter(strings):    
    my_counter = {}
    for line in strings:
        for word in line:
            my_counter[word] = my_counter.get(word, 0) + 1
    return my_counter

In [326]:
#Extracting only the words (which corresponds to the dictionary keys)
def unique_words(dictionary):    
    comments_words=[]
    for item in dictionary.keys():
        comments_words.append(item)
    return comments_words

In [327]:
def wordset(querydoc,commentsdoc):
    wordset=set(querydoc).union(set(commentsdoc))
    return wordset

In [328]:
#Initialization with Zeros
def tf_raw_query(query_strings,wordset):    
    worddict_query=dict.fromkeys(wordset,0)    
    for word in query_strings:
        worddict_query[word]+=1
    return worddict_query

In [329]:
#Calculation of the tf-weight terms
def tf_weight(words_raw):    
    dict_tf_wt={}
    import math
    for key,val in words_raw.items():
        if val!=0:
            value=1+(math.log(val))
        else:
            value=0
        dict_tf_wt[key]=value
    return dict_tf_wt

In [330]:
#Find the documents frequency - Number of documents a term exist
def df_df(wordset,doc):   
    dict_df={}
    for item in wordset:
        dfg=0
        for item2 in doc:
            if item in item2:
                dfg+=1            
            else:
                dfg=0
            dict_df[item]=dfg
    return dict_df

In [331]:
#Calculating the idf for a query
def idf_query(df):
    idf=[]
    import math
    for item in df['df']:
        if item !=0:
            value=math.log(10/item)
        else:
            value=0
        idf.append(value)
    df["idf"]=idf
    return df

In [332]:
#Normalizing the weights
def norm_q(df):    
    sum1=0.0
    for item in df['wt']:
        sq=item*item
        sum1+=sq
    import math
    norm_q=[]
    for itemm in df['wt']:
        if item==0:
            deno=0
        else:
            deno=itemm/math.sqrt((sum1))
        norm_q.append(deno)
    df["nor_q"]=norm_q
    return df

In [333]:
def log_wt(val):
    import math
    if val!=0:
        value=1+(math.log(val))
    else:
        value=0
    return val

#### Importing the Question-Comments XML file,Identifying the Question-Comments within the XML file

In [334]:
#Importing the XML file and required libraries
import xml.etree.ElementTree as ET
import pandas as pd
tree = ET.parse('small_train_1.xml')
root = tree.getroot()

In [335]:
#Extracting the Questions and comments
#Question
for thread in root.iter("Thread"):
    for question in root.iter('RelQuestion'):
        qa=question.attrib['RELQ_ID']
        query=""
        for question in root.iter('RelQBody'):
            each_question=question.text
            query+=each_question

            #Tokenization of the Question
            hey=tokenization(query)

            #Removing the stop words from the Question
            sq=stop_words(hey)

            #Stemming the words in Question
            st=stemmer(sq)

            #Comments
            comments=[]
            ids=[]
        for each_answer in root.iter('RelCText'):
            answer=each_answer.text
            comments.append(answer)
            for comment in root.iter('RelComment'):
                relc_id=comment.attrib['RELC_ID']
                ids.append(relc_id)

        result=pd.DataFrame()
            #Removing stop words from the Comments    
        sc_co=sw_co(comments)

            #Stemming the words in the comments
        st_co=stemmer_co(sc_co)

            #Creating a wordset for the comments
        counter1=word_counter(st_co)
        unique_words1=unique_words(counter1)
        wordset1=wordset(st,unique_words1)

            #Calculating the raw terms in the comments
        tf_raw=tf_raw_query(st,wordset1)
        tf_wt=tf_weight(tf_raw)
        df_query=df_df(wordset1,st_co)

            #Appending the tf-raw, tf-wt, df into a dataframe

        dframe=pd.DataFrame.from_dict([tf_raw,tf_wt,df_query],orient='columns')
        df=dframe.transpose()
        summary_of_results=df.rename(columns={0: 'tf_raw',1: 'tf-weight',2: 'df'})

            #Calculating the IDF for the query
        idf_query=idf_query(summary_of_results)
        idf_query["wt"]=idf_query.idf*idf_query.tf_raw

            #Normalization
        norm_query=norm_q(idf_query)

            #Calculating the raw-terms for all the comments

        for item in st_co:
            worddict_terms=dict.fromkeys(wordset1,0)  
            frames=pd.DataFrame()
            for items in item:
                worddict_terms[items]+=1
                df_com_c1=pd.DataFrame.from_dict([worddict_terms])
            frames=[result,df_com_c1]
            result = pd.concat(frames)
        hey=result.transpose()

            #Calculating the terms weight
        hey_1=hey.applymap(log_wt)

            #Adding the column name to the Dataframe
        hey_1.columns=["tf_doc_wt_1","tf_doc_wt_2","tf_doc_wt_3","tf_doc_wt_4","tf_doc_wt_5","tf_doc_wt_6","tf_doc_wt_7","tf_doc_wt_8","tf_doc_wt_9",
                           "tf_doc_wt_10"]


            #Normalization of all the comments weights
        counter=1
        for item in hey_1:
            sum2=0.0
            for items in hey_1[item]:
                sq1=items*items
                sum2+=sq1
                import math
                norm_q_d=[]
                for itemw in hey_1[item]:
                    if itemw==0:
                        deno=0
                    else:
                        deno=itemw/math.sqrt((sum2))
                    norm_q_d.append(deno)
            s=str(counter)
            hey_1[s]=norm_q_d
            counter+=1

            #Keeping the column weights 
        cols = [0,1,2,3,4,5,6,7,8,9]
        hey_1.drop(hey_1.columns[cols],axis=1,inplace=True)

            #product
        columns = hey_1.columns[:]    
        for item in columns:
            if item !=0:
                hey_1[columns] *= norm_query['nor_q']
            else:
                hey_1[columns]=0

            #Replacing the NaN values with Zero's
        hey_2=hey_1.fillna('0')

            #Query - Documents score
        Cosine_Score=hey_2.sum(axis=0)       


C:\Users\Jessica\Miniconda3\envs\py35\lib\site-packages\ipykernel\__main__.py:93: RuntimeWarning: divide by zero encountered in true_divide


In [336]:
Cosine_Score

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
dtype: float64